In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

## Langsmith Tracking and tracing

os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [ ]:
from langchain_openai import ChatOpenAI

llm=ChatOpenAI(model="o1-mini")

client=<openai.resources.chat.completions.Completions object at 0x000002510F21F200> async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000002510F256D20> root_client=<openai.OpenAI object at 0x000002510EE08110> root_async_client=<openai.AsyncOpenAI object at 0x000002510F2550A0> model_name='o1-mini' temperature=1.0 model_kwargs={} openai_api_key=SecretStr('**********')


In [ ]:
result=llm.invoke("What is agentic AI")

content="**Agentic AI** refers to artificial intelligence systems designed to operate as autonomous agents capable of making independent decisions, taking actions, and pursuing defined goals within an environment. The concept emphasizes the AI's ability to exhibit agency—the capacity to act intentionally and adaptively—much like a human or other autonomous entity would.\n\n### Key Characteristics of Agentic AI\n\n1. **Autonomy**: Agentic AI systems can operate without continuous human oversight, making decisions based on their programming, learning, and interactions with the environment.\n\n2. **Goal-Oriented Behavior**: These systems are designed with specific objectives and can plan and execute actions to achieve these goals.\n\n3. **Adaptability and Learning**: Agentic AI can learn from experiences, adapt to new situations, and improve performance over time through techniques like machine learning.\n\n4. **Perception and Interaction**: They can perceive their environment through sen

In [5]:
print(result.content)

**Agentic AI** refers to artificial intelligence systems designed to operate as autonomous agents capable of making independent decisions, taking actions, and pursuing defined goals within an environment. The concept emphasizes the AI's ability to exhibit agency—the capacity to act intentionally and adaptively—much like a human or other autonomous entity would.

### Key Characteristics of Agentic AI

1. **Autonomy**: Agentic AI systems can operate without continuous human oversight, making decisions based on their programming, learning, and interactions with the environment.

2. **Goal-Oriented Behavior**: These systems are designed with specific objectives and can plan and execute actions to achieve these goals.

3. **Adaptability and Learning**: Agentic AI can learn from experiences, adapt to new situations, and improve performance over time through techniques like machine learning.

4. **Perception and Interaction**: They can perceive their environment through sensors (e.g., cameras

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are an expert AI Engineer. Provide me answer based on the question"),
        ("user","{input}")

    ]
)

In [6]:
llm=ChatOpenAI(model="gpt-4o")

In [ ]:
chain=prompt|llm 

response=chain.invoke({"input":"Can you tell me about Langsmith"})

In [ ]:
print(response.content)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
output_parser=StrOutputParser()

chain=prompt|llm|output_parser

response=chain.invoke({"input":"Can you tell me about Langsmith?"})
print(response)

In [27]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser


output_parser=JsonOutputParser()
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)


In [ ]:
chain=prompt|llm|output_parser

response=chain.invoke({"query":"Can you tell me about Langsmith?"})
print(response)

### RAG

In [30]:
from langchain_community.document_loaders import WebBaseLoader

In [ ]:
loader=WebBaseLoader("https://python.langchain.com/docs/tutorials/llm_chain/")
loader

In [ ]:
document=loader.load()
document

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(document)
documents

In [34]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [35]:
from langchain_community.vectorstores import FAISS

vectorstore=FAISS.from_documents(documents,embeddings)
vectorstore

In [ ]:
query="This is a relatively simple LLM application "

result=vectorstore.similarity_search(query)
result[0].page_content

In [36]:
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

In [ ]:
prompt

In [ ]:
# pip install -U langchain langchain-community

from langchain_community.chat_models import ChatOpenAI
from langchain_core.documents import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_messages(
    [("system", "What are everyone's favorite colors:\n\n{context}")]
)
llm = ChatOpenAI(model="gpt-3.5-turbo")
chain = create_stuff_documents_chain(llm, prompt)

chain

In [25]:
docs = [
    Document(page_content="Jesse loves red but not yellow"),
    Document(page_content = "Jamal loves green but not as much as he loves orange")
]

chain.invoke({"context": docs})

'\n\nJesse: Red\nJamal: Green, Orange'

In [ ]:
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt)

In [ ]:
vectorstore.similarity_search("Note that ChatModels receive message objects as inpu")

In [ ]:
document_chain.invoke({
    "input":"Note that ChatModels receive message objects as input",
    "context":[Document(page_content="Note that ChatModels receive message objects as input and generate message objects as output. In addition to text content, message objects convey conversational roles and hold important data, such as tool calls and token usage counts.\nLangChain also supports chat model inputs via strings or OpenAI format. The following are equivalent:\nmodel.invoke('Hello')model.invoke([{'role': 'user', 'content': 'Hello'}])model.invoke([HumanMessage('Hello')]) StreamingBecause chat models are Runnables, they expose a standard interface that includes async and streaming modes of invocation. This allows us to stream individual tokens from a chat model:\nfor token in model.stream(messages):    print(token.content, end='|')|C|iao|!|| You can find more details on streaming chat model outputs in this guide.\nPrompt Templates")]
})

In [ ]:
retriever=vectorstore.as_retriever()

from langchain.chains import create_retrieval_chain

retrieval_chain=create_retrieval_chain(retriever,document_chain)
retrieval_chain

In [46]:
result=retrieval_chain.invoke({"input":"Note that ChatModels receive message objects as input"})

In [ ]:
result['answer']